In [1]:
#trace_directory = '~/.ros/tracing/profile-ros-20190812-134428'
#trace_directory = '~/lttng-traces/profile-test-20190812-102605'
#trace_directory = '~/lttng-traces/profile-ros-20190812-134428'
trace_directory = '~/lttng-traces/profile-ros-instr-overall-20190813-110937'

In [2]:
import sys
# Assuming a workspace with:
#   src/tracetools_analysis/
#   src/micro-ROS/ros_tracing/ros2_tracing/tracetools_read/
sys.path.insert(0, '../')
sys.path.insert(0, '../../../micro-ROS/ros_tracing/ros2_tracing/tracetools_read/')
import datetime as dt
import os

from bokeh.palettes import Category20
from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import PrintfTickFormatter
from bokeh.models.widgets import Div
import numpy as np
import pandas as pd

from tracetools_analysis import utils
from tracetools_analysis.conversion import ctf
from tracetools_analysis.loading import load_pickle
from tracetools_analysis.processor import Processor
from tracetools_analysis.processor.cpu_time import CpuTimeHandler
from tracetools_analysis.processor.profile import ProfileHandler
from tracetools_analysis.processor.ros2 import Ros2Handler

In [3]:
# Convert
trace_directory = os.path.expanduser(trace_directory)
pickle_filename = 'pickle'
pickle_path = os.path.join(trace_directory, pickle_filename)
count = ctf.convert(trace_directory, pickle_path)
print(f'{count} events')

288796 events


In [4]:
# Process
events = load_pickle(pickle_path)
address_to_func = {
    '': 'spin',
    '': 'get_next_executable',
    '': 'get_next_ready_executable',
    '': 'wait_for_work',
    '': 'execute_any_executable',
}
profile_handler = ProfileHandler(address_to_func=address_to_func)
ros_handler = Ros2Handler()
cpu_handler = CpuTimeHandler()
processor = Processor(profile_handler, ros_handler, cpu_handler)
processor.process(events)

In [5]:
profile_util = utils.ProfileDataModelUtil(profile_handler.data)
ros_util = utils.RosDataModelUtil(ros_handler.data)
cpu_util = utils.CpuTimeDataModelUtil(cpu_handler.data)

output_notebook()
psize = 450

tids = profile_util.get_tids()
print(tids)

Loading BokehJS ...

{8116}


In [6]:
# Select the first one, since we only have one in this case anyway
tid = list(tids)[0]

In [43]:
node_names = ros_util.get_node_names_from_tid(tid)
node_names_format = ', '.join(node_names) if node_names is not None else 'unknown'
# Convert time columns
utils.DataModelUtil.convert_time_columns(
    profile_handler.data.times,
    ['duration', 'actual_duration'],
    ['start_timestamp'],
)
# Plot durations
functions_data = profile_util.get_function_duration_data(tid)
for function_data in functions_data:
    depth = function_data['depth']
    name = function_data['function_name']
    parent = function_data['parent_name']
    df = function_data['data']

    # Duration vs. actual duration
    starttime = df['start_timestamp'].iloc[0].strftime('%Y-%m-%d %H:%M')
    duration_source = ColumnDataSource(df)
    duration = figure(
        title=f'Function duration over time',
        x_axis_label=f'start ({starttime})',
        y_axis_label='duration (ms)',
        plot_width=psize, plot_height=psize,
    )
    duration.title.align = 'center'
    duration.varea_stack(
        ['actual_duration', 'duration_difference'],
        x='start_timestamp',
        color=('blue', 'red'),
        legend=('ON CPU', 'OFF CPU'),
        source=duration_source,
    )
    duration.legend.label_text_font_size = '11px'
    duration.xaxis[0].formatter = DatetimeTickFormatter(seconds=['%Ss'])

    # Histogram
    dur_hist_overall, edges_overall = np.histogram(df['duration'], bins=10)
    dur_hist_actual, edges_actual = np.histogram(df['actual_duration'], bins=10)
    duration_hist_overall = pd.DataFrame({
        'duration': dur_hist_overall, 
        'left': edges_overall[:-1], 
        'right': edges_overall[1:],
    })
    duration_hist_actual = pd.DataFrame({
        'actual_duration': dur_hist_actual, 
        'left': edges_actual[:-1], 
        'right': edges_actual[1:],
    })
    histogram = figure(
        title=f'Distribution',
        x_axis_label='duration (ms)',
        y_axis_label='frequency',
        plot_width=psize, plot_height=psize,
    )
    histogram.title.align = 'center'
    histogram.quad(
        bottom=0, top=duration_hist_overall['duration'],
        left=duration_hist_overall['left'], right=duration_hist_overall['right'],
        legend='overall duration',
        color='blue',
    )
    histogram.quad(
        bottom=0, top=duration_hist_actual['actual_duration'],
        left=duration_hist_actual['left'], right=duration_hist_actual['right'],
        legend='ON CPU',
        color='red',
    )

    show(column(Div(text=f'<h2>{name}()</h2>'),
                row(duration, histogram)))

# Comparison
colors = Category20[20]

data = {
    'depth': ['1'],
}
elements = []
legend = []
for function_data in functions_data:
    depth = function_data['depth']
    if depth != 1:
        continue
    name = function_data['function_name']
    actual_duration_mean = function_data['data']['actual_duration'].mean()
    duration_mean_diff = function_data['data']['duration_difference'].mean()
    actual_string = f'{name}, actual'
    diff_string = f'{name}, duration-actual'
    data[actual_string] = [actual_duration_mean]
    data[diff_string] = [duration_mean_diff]
    elements.append(actual_string)
    elements.append(diff_string)
    legend.append(f'{name}() ON CPU: {actual_duration_mean:.4f}')
    legend.append(f'{name}() OFF CPU: {duration_mean_diff:.4f}')

comparison = figure(
    y_range=['1'],
    title=f'Mean durations comparison',
    x_axis_label='mean duration (ms)',
    y_axis_label='depth',
    tooltips='$name: @$name{1.11}',
    plot_width=psize+200, plot_height=psize,
)
comparison.hbar_stack(
    elements, y='depth', height=0.3,
    color=colors[:(len(elements))],
    source=data,
    legend=legend,
)
comparison.title.align = 'center'
comparison.legend.label_text_font_size = '10px'
show(comparison)


TypeError: ("unsupported operand type(s) for /: 'Timestamp' and 'float'", 'occurred at index start_timestamp')

In [48]:
# CPU time

# for the whole process/thread
time_per_thread = cpu_util.get_time_per_thread()
times_tid = cpu_handler.data.times[cpu_handler.data.times['tid'] == tid]
begin = times_tid.iloc[0]['start_timestamp']
end = times_tid.iloc[-1]['start_timestamp'] + times_tid.iloc[-1]['duration']
overall_thread_duration = (end - begin) / (1000000.0 * 1000.0)
actual_thread_duration = time_per_thread.loc[tid, 'duration'] / (1000000.0 * 1000.0)
thread_percent = 100.0 * (actual_thread_duration / overall_thread_duration)
print('Whole thread')
print(f'overall_thread_duration: {overall_thread_duration:.2f} s')
print(f'actual_thread_duration : {actual_thread_duration:.2f} s  {thread_percent:.2f} %')

print()
# for wait_for_work()
function_data_waitforwork = next(
    (data for data in functions_data if data['depth'] == 0 and data['function_name'] == 'wait_for_work'),
    None,
)
assert function_data_waitforwork is not None
data = function_data_waitforwork['data']
overall_duration_waitforwork = (data['duration'].sum() / 1000.0)
actual_duration_waitforwork = (data['actual_duration'].sum() / 1000.0)
function_percent_waitforwork = 100.0 * (actual_duration_waitforwork / overall_duration_waitforwork)
print('wait_for_work() function')
print(f'overall_duration       : {overall_duration_waitforwork:.2f} s')
print(f'actual_duration        : {actual_duration_waitforwork:.2f} s  {function_percent_waitforwork:.2f} %')

# for get_next_ready_executable()
function_data_getnextreadyexecutable = next(
    (data for data in functions_data if data['depth'] == 0 and data['function_name'] == 'get_next_ready_executable'),
    None,
)
assert function_data_getnextreadyexecutable is not None
data = function_data_getnextreadyexecutable['data']
overall_duration_getnextreadyexecutable = (data['duration'].sum() / 1000.0)
actual_duration_getnextreadyexecutable = (data['actual_duration'].sum() / 1000.0)
function_percent_getnextreadyexecutable = 100.0 * (actual_duration_getnextreadyexecutable / overall_duration_getnextreadyexecutable)
print('get_next_ready_executable() function')
print(f'overall_duration       : {overall_duration_getnextreadyexecutable:.2f} s')
print(f'actual_duration        : {actual_duration_getnextreadyexecutable:.2f} s  {function_percent_getnextreadyexecutable:.2f} %')

actual_duration_total = actual_duration_waitforwork + actual_duration_getnextreadyexecutable
print('total:', actual_duration_total)

Whole thread
overall_thread_duration: 4.58 s
actual_thread_duration : 3.63 s  79.11 %

wait_for_work() function
overall_duration       : 1.08 s
actual_duration        : 0.55 s  50.74 %
get_next_ready_executable() function
overall_duration       : 2.60 s
actual_duration        : 2.53 s  97.25 %
total: 3.077137852


In [45]:
# Comparison
colors = Category20[20]

data = {
    'compared to': ['thread', 'function'],
}
elements = []
legend = []
for function_data in functions_data:
    depth = function_data['depth']
    if depth != 1:
        continue
    name = function_data['function_name']
    actual_duration_mean = function_data['data']['actual_duration'].sum() / 1000.0
    actual_duration_percentage_thread = 100.0 * (actual_duration_mean / actual_thread_duration)
    actual_duration_percentage_function = 100.0 * (actual_duration_mean / actual_duration_waitforwork)
    actual_string_thread = f'{name}, wrt thread'
    actual_string_function = f'{name}, wrt function'
    data[actual_string_thread] = [actual_duration_percentage_thread, 0.0]
    data[actual_string_function] = [0.0, actual_duration_percentage_function]
    elements.append(actual_string_thread)
    elements.append(actual_string_function)
    legend.append(f'{name}() wrt thread ON CPU: {actual_duration_percentage_thread:.2f} %')
    legend.append(f'{name}() wrt function ON CPU: {actual_duration_percentage_function:.2f} %')

comparison = figure(
    y_range=['function', 'thread'],
    title=f'Total duration comparison (wrt whole thread & wait_for_work() function)',
    x_axis_label='proportion (%)',
    y_axis_label='compared to',
    tooltips='$name: @$name{1.11}',
    plot_width=psize+400, plot_height=psize,
)
comparison.hbar_stack(
    elements, y='compared to', height=0.3,
    color=colors[:(len(elements))],
    source=data,
    legend=legend,
)
comparison.title.align = 'center'
comparison.title.text_font_size = '20px'
comparison.legend.label_text_font_size = '10px'
show(comparison)

In [51]:
# Comparison
colors = Category20[20]

data_ = {
    'compared to': ['thread', 'functions'],
}
elements = []
legend = []
for function_data in functions_data:
    depth = function_data['depth']
    if depth != 0:
        continue
    name = function_data['function_name']
    actual_duration_mean = function_data['data']['actual_duration'].sum() / 1000.0
    actual_duration_percentage_thread = 100.0 * (actual_duration_mean / actual_thread_duration)
    actual_duration_percentage_function = 100.0 * (actual_duration_mean / actual_duration_total)
    actual_string_thread = f'{name}, wrt thread'
    actual_string_function = f'{name}, wrt functions'
    data_[actual_string_thread] = [actual_duration_percentage_thread, 0.0]
    data_[actual_string_function] = [0.0, actual_duration_percentage_function]
    elements.append(actual_string_thread)
    elements.append(actual_string_function)
    legend.append(f'{name}() wrt thread ON CPU: {actual_duration_percentage_thread:.2f} %')
    legend.append(f'{name}() wrt functions ON CPU: {actual_duration_percentage_function:.2f} %')

comparison = figure(
    y_range=['functions', 'thread'],
    title=f'Total duration comparison (wrt whole thread & get_next_ready_executable()+wait_for_work() functions)',
    x_axis_label='proportion (%)',
    y_axis_label='compared to',
    tooltips='$name: @$name{1.11}',
    plot_width=psize+400, plot_height=psize,
)
comparison.hbar_stack(
    elements, y='compared to', height=0.3,
    color=colors[:(len(elements))],
    source=data_,
    legend=legend,
)
comparison.title.align = 'center'
comparison.title.text_font_size = '15px'
comparison.legend.label_text_font_size = '10px'
show(comparison)